In [ ]:
!pip install langchain_openai
!pip install langchain-community
!pip install pypdf
!pip install faiss-cpu

# 1. 사용 환경 준비

In [10]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

# 2. 모델 초기화

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

# 3. 문서 로드

In [ ]:
import json
import os
from langchain.document_loaders.base import BaseLoader
from langchain.schema import Document

class MultiJSONCaseLoader(BaseLoader):
    def __init__(self, folder_path: str):
        self.folder_path = folder_path
    
    def load(self):
        documents = []
        # 폴더 내의 모든 JSON 파일을 순회
        for filename in os.listdir(self.folder_path):
            if filename.endswith('.json'):
                file_path = os.path.join(self.folder_path, filename)
                try:
                    with open(file_path, "r", encoding="utf-8") as file:
                        case_data = json.load(file)
                        # 단일 객체인 경우 리스트로 변환
                        if not isinstance(case_data, list):
                            case_data = [case_data]
                        
                        for case in case_data:
                            content = f"""
                            판례일련번호: {case.get("판례일련번호", "")}
                            사건명: {case.get("사건명", "")}
                            사건번호: {case.get("사건번호", "")}
                            선고일자: {case.get("선고일자", "")}
                            법원명: {case.get("법원명", "")}
                            사건종류명: {case.get("사건종류명", "")}
                            판결요지: {case.get("판결요지", "")}
                            판례내용: {case.get("판례내용", "")}
                            """
                            documents.append(Document(page_content=content, metadata=case))
                        print(f"Loaded {len(case_data)} cases from {filename}")
                except Exception as e:
                    print(f"Error loading {filename}: {str(e)}")
                    continue
        
        return documents

# 폴더 경로를 사용하여 로더 초기화
loader = MultiJSONCaseLoader("/Users/ohhalim/Desktop/civilprecedent")

# 문서 데이터 로드
docs = loader.load()

# 로드 결과 확인
print(f"\nTotal documents loaded: {len(docs)}")
print("\nSample document:")
if docs:
    print(docs[0].page_content[:500] + "...")

# 4. chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm  # 진행상황 확인용

# 배치 사이즈 설정
BATCH_SIZE = 100

# 텍스트 스플리터 설정
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 배치 처리 함수
def process_in_batches(docs):
    all_splits = []
    total_batches = len(docs) // BATCH_SIZE + (1 if len(docs) % BATCH_SIZE else 0)
    
    for i in tqdm(range(0, len(docs), BATCH_SIZE)):
        batch = docs[i:i + BATCH_SIZE]
        batch_splits = recursive_text_splitter.split_documents(batch)
        
        # 결과 확인 (첫 번째 배치만)
        if i == 0:
            print(f"\n첫 번째 배치 예시:")
            print(f"첫 번째 split: {batch_splits[0]}")
            print(f"이 배치의 split 수: {len(batch_splits)}")
        
        all_splits.extend(batch_splits)
        
        # 메모리 사용량 출력 (선택사항)
        # import psutil
        # print(f"Memory usage: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB")
    
    return all_splits

# 실행
splits = process_in_batches(docs)
print(f"\n총 split 수: {len(splits)}")

# 5. embedding

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") # 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성

# 6. vector store 생성

In [15]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 7. retriever 생성

In [16]:
from langchain.vectorstores.base import VectorStore

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 8. 프롬프트 템플릿 정의

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context. Please provide the case number of a similar case. "),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

# 9. RAG 체인 구성

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

'''
RunnablePassthrough를 상속받은 DebugPassThrough 클래스
LangChain에서 RunnablePassthrough가 기본적으로 "데이터를 그대로 전달"하는 역할
잘 전달하는지 출력하기 위해 만들어진 커스텀 츨래스입니다.
'''
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs) # 부모 클래스의 invoke 호출. 부모클래스의 invoke는 그냥 전달받은걸 그대로 전달한다.
        print("Debug Output:", output)
        return output


'''
RunnablePassthrough를 상속받은 ContextToText 클래스
context와 question를 정리하여 리턴합니다.
'''
class ContextToText(RunnablePassthrough):
      def invoke(self, inputs, config=None, **kwargs):
        if isinstance(inputs["context"], list) :
            context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        else:
            context_text = inputs["context"]
        return {"context": context_text, "question": inputs["question"]}


'''
정의된 순차적으로 진행되는 rag chain
'''
rag_chain_debug = {
    "context": retriever,  # invoke(query)가 실행되면 제일 먼저 리트리버가 실행된다. retriever 실행결과는 "context" 키에 저장된다.
    "question": DebugPassThrough() # query는 "question" 키에 저장된다. DebugPassThrough(query를)를 실행한다.
}  | DebugPassThrough() | ContextToText() | contextual_prompt | model
# DebugPassThrough() : 위에서 리턴받은 {"context", "question"} 쌍을 입력으로 받고 그대로 출력하고 리턴한다.
# ContextToText() : 위에서 리턴받은 {"context", "question"} 쌍을 ContextToText()에 넣고 정리하여 리턴한다.
# contextual_prompt : 위에서 리턴받은 정리된 {"context", "question"} 쌍을 프롬프트 템플릿에 넣고 프롬프트로 만들어 리턴한다.
# model : 위에서 리턴받은 프롬프트를 모델에 넣고 모델의 출력을 리턴한다

In [ ]:
print("========================")
query = input("질문을 입력하세요: ")
response = rag_chain_debug.invoke(query)
print("Final Response:")
print(response.content)

# FAQ

### Q. contextual_prompt, model 등은 그냥 변수명일 뿐인데 어떻게 체인 안에 들어가나요?

- contextual_prompt은 '템플릿'을 부르는 변수명입니다. 해당 템플릿을 살펴보면 분명 intput과 output이 있음을 알 수 있습니다.

input :
{context} : 사칙 3873조. 우리 회사의 점심 시간은 12:30 ~ 1:30 입니다.
사칙 96887858483조. 우리 회사의 석식 시간은 18:00 ~ 19:00 입니다.
{question} :우리 회사의 사내 밥 시간 규정은 어떻게 돼?

```
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])
```

output :
"Answer the question using only the following context."
"Context" : 사칙 3873조. 우리 회사의 점심 시간은 12:30 ~ 1:30 입니다.
사칙 96887858483조. 우리 회사의 석식 시간은 18:00 ~ 19:00 입니다."
"Question" : "우리 회사의 사내 밥 시간 규정은 어떻게 돼?"

- model 또한 당연히 input, output이 있는 객체이며 내부적으로 invoke()가 일어난다고 생각하시면 됩니다.




### Q. 다른 chain 구조 예시는 없나요?


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 텍스트로부터 FAISS 벡터 저장소를 생성
vectorstore = FAISS.from_texts(
    [
        "우리 회사의 석식시간은 18:00시부터입니다.",
        "우리 회사의 중식시간은 12:00시부터입니다.",
        "우리 회사의 야근 식대는 2만원정입니다.",
        "17층 복도 마지막 회의실 소등시 누군가 부르면 뒤돌아보면 안됩니다.",
    ],
    embedding=OpenAIEmbeddings(),
)
# 리트리버 객체 생성
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
{context}

Question: {question}
""")

# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# rag 체인 구성
retrieval_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()} # RunnablePassthrough() : 있는 그대로 리턴하는 함수
    | prompt
    | model
    | StrOutputParser()
)

# rag 체인 실행
retrieval_chain.invoke("우리 회사의 야근 식대는 얼마입니까?") # 답: 2만원정
